<a href="https://colab.research.google.com/github/Kaiziferr/NLP_Workshop/blob/master/w0_word_embedding_layersipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La capa de incrustación se inicializa con pesos aleatorios y aprenderá una incrustación para todas las palabras en el conjunto de datos de entrenamiento.

- Se puede usar solo para aprender una incrustación de palabras que se puede guardar y usar en otro modelo más adelante.
-Se puede utilizar como parte de un modelo de aprendizaje profundo en el que la incorporación se aprende junto con el modelo en sí
- Se puede utilizar para cargar un modelo de incrustación de palabras previamente entrenado, un tipo de aprendizaje por transferencia.

Argumentos
- input_dim: tamaño del vocabulario en los datos de texto Por ejemplo, si sus datos están codificados en números enteros con valores entre 0 y 10, entonces el tamaño del vocabulario sería de 11 palabras.
- output_dim: este es el tamaño del espacio vectorial en el que se incrustarán las palabras.
- input_length : Esta es la longitud de las secuencias de entrada, como definiría para cualquier capa de entrada de un modelo de Keras. Por ejemplo, si todos sus documentos de entrada están compuestos por 1000 palabras, esto sería 1000.

In [4]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.layers.embeddings import Embedding

from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences



import numpy as np


In [5]:
docs = ['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!', 'Weak', 'Poor effort!', 'not good', 'poor work', 'Could have done better.']
labels = np.array([1,1,1,1,1,0,0,0,0,0])
vocab_size = 50

In [6]:
# One-hot codifica un texto en una lista de índices de palabras de tamaño n
encoded_docs = [one_hot(d, vocab_size, split=' ') for d in docs]
encoded_docs

[[8, 31],
 [18, 25],
 [26, 18],
 [9, 25],
 [9],
 [22],
 [8, 18],
 [25, 18],
 [8, 25],
 [14, 31, 31, 45]]

In [7]:
sequence = [[1], [2, 3], [4, 5, 6], [7,8,9,10]]
pad_sequences(sequence, padding='post', maxlen=2, dtype='float64', truncating='pre')

array([[ 1.,  0.],
       [ 2.,  3.],
       [ 5.,  6.],
       [ 9., 10.]])

In [8]:
mx_length = 4
padded_doc = pad_sequences(encoded_docs, maxlen=mx_length, padding='post', value=0)
print(padded_doc, type(padded_doc), padded_doc.shape, padded_doc.ndim)

[[ 8 31  0  0]
 [18 25  0  0]
 [26 18  0  0]
 [ 9 25  0  0]
 [ 9  0  0  0]
 [22  0  0  0]
 [ 8 18  0  0]
 [25 18  0  0]
 [ 8 25  0  0]
 [14 31 31 45]] <class 'numpy.ndarray'> (10, 4) 2


In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length = mx_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(padded_doc, labels, epochs=50, verbose=0)

In [12]:
loss, accuracy = model.evaluate(padded_doc, labels,verbose=0)
print('Accuracy %f' % (accuracy*100))

Accuracy 69.999999


In [13]:
doc_valida = ['Well work']
input_valide_doc = [one_hot(doc_valida[0], vocab_size, split=' ')]
padded_doc_valid = pad_sequences(input_valide_doc, padding='post', maxlen = 4, value=0)
padded_doc_valid

array([[ 8, 25,  0,  0]], dtype=int32)

In [14]:
model.predict(padded_doc_valid)

array([[0.5156619]], dtype=float32)

Keras proporciona una clase Tokenizer que se puede ajustar a los datos de entrenamiento, puede convertir texto en secuencias de manera consistente llamando al método texts_to_sequences () en la clase Tokenizer , y brinda acceso al mapeo del diccionario de palabras a enteros en un atributo word_index .

In [15]:
t = Tokenizer()
t.fit_on_texts(docs)
t.word_index

{'better': 14,
 'could': 12,
 'done': 2,
 'effort': 4,
 'excellent': 9,
 'good': 3,
 'great': 7,
 'have': 13,
 'nice': 8,
 'not': 11,
 'poor': 5,
 'weak': 10,
 'well': 6,
 'work': 1}

In [16]:
vocab_size = len(t.word_index) + 1
vocab_size

15

In [17]:
encoded_docs = t.texts_to_sequences(docs)
encoded_docs

[[6, 2],
 [3, 1],
 [7, 4],
 [8, 1],
 [9],
 [10],
 [5, 4],
 [11, 3],
 [5, 1],
 [12, 13, 2, 14]]

In [18]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [23]:
embeddings_index = dict()
f = open('./data/glove.6B.100d.txt')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.array(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 39103 word vectors.


In [29]:
embedding_matrix = np.zeros((vocab_size,100))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [34]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.11619   ,  0.45447001, -0.69216001, ..., -0.54737002,
         0.48822001,  0.32246   ],
       [-0.2978    ,  0.31147   , -0.14937   , ..., -0.22709   ,
        -0.029261  ,  0.4585    ],
       ...,
       [ 0.05869   ,  0.40272999,  0.38633999, ..., -0.35973999,
         0.43718001,  0.10121   ],
       [ 0.15711001,  0.65605998,  0.0021149 , ..., -0.60614997,
         0.71004999,  0.41468999],
       [-0.047543  ,  0.51914001,  0.34283999, ..., -0.26859   ,
         0.48664999,  0.55609   ]])

In [35]:
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None


In [44]:
model.fit(padded_docs, labels, epochs=50, verbose=0)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


# Referencias
- https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/